![img](https://drive.google.com/uc?export=view&id=1kgX98Ziw9LzgBCT2BxChttfrTcBltLej)


# MÓDULO 3. Unidad 2 - Clase 1

---

> Redes Neuronales Recurrentes.

> Red Neuronal Recurrente Simple en Keras. Funcionamiento.

> Ejemplo de Aplicación: Predicción del Precio de un Activo.

> Otros Tipos Más Sofisticados de RNN

# Redes Neuronales Recurrentes (RNN, Recurrent Neural Networks)

Las **redes neuronales recurrentes** son un tipo particular de red neuronal que agrega recurrencia a la relación entre las neuronas de una capa. Esta recurrencia es lo que introduce una especie de "memoria" a la topología de la red.

![img](https://drive.google.com/uc?export=view&id=14CVwzArMr-lbnICRt5HI-9K7GL80K10h)

Las redes recurrentes son utilizadas para resolver problemas de predicción de secuencias, como por ejemplo problemas de predicción de series temporales, ya que le podemos pasar una cantidad $n$ de valores (o *steps*) para predecir el valor $n+1$ (próximo *step*). También se usan para predecir la próxima palabra, dada una secuencia de palabras. Les suena esto?

Las aplicaciones y contenidos de esta notebook están basados en el siguiente post: https://machinelearningmastery.com/understanding-simple-recurrent-neural-networks-in-keras/

# Red Neuronal Recurrente Simple en **Keras**

Veamos cómo podemos crear una RNN usando Keras.

## Importaciones

In [ ]:
# Correr si es necesario, para instalar Yahoo Finance
!pip install yfinance

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt
import yfinance as yf

In [ ]:
import tensorflow as tf
import random as python_random

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(123)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
python_random.seed(123)

# The below set_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/random/set_seed
tf.random.set_seed(123)

## Red Neuronal Recurrente Simple

Con la siguiente función, creamos un modelo usando la librería **Keras** que incluye una capa `SimpleRNN` y una capa `Dense`(densa) para aprender datos secuenciales.

In [ ]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, 
                        activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

demo_model = create_RNN(2, 1, (3,1), activation=['linear', 'linear'])

El `input_shape` se establece en 3×1 y, para simplificar, se usa la activación lineal para ambas capas ($f(x) = x$).

Entonces, si tenemos $m$ unidades ocultas (`hidden_units`, en este caso $m=2$), la red tiene la siguiente pinta:

- Input: $x \in R$
- Hidden unit (Unidades ocultas): $h \in R^m$
- Weights o pesos para las input units: $w_x \in R^m$
- Weights o pesos para las hidden units: $w_h \in R^{mxm}$
- Bias o sesgos para las hidden units: $b_h \in R^m$
- Weight o pesos para la *dense layer* (capa densa): $w_y \in R^m$
- Bias o sesgo para la *dense layer* (capa densa): $b_y \in R$

Veamos estos pesos en nuestro modelo:

In [ ]:
wx = demo_model.get_weights()[0]
wh = demo_model.get_weights()[1]
bh = demo_model.get_weights()[2]
wy = demo_model.get_weights()[3]
by = demo_model.get_weights()[4]

print('wx = ', wx)
print('shape = ', wx.shape)
print()
print('wh = ', wh)
print('shape = ', wh.shape)
print()
print('bh = ', bh)
print('shape = ', bh.shape)
print()
print('wy =', wy)
print('shape = ', wy.shape)
print()
print('by = ', by)
print('shape = ', by.shape)

Podemos analizar la estructura de este modelo a partir de la siguiente imagen:

![img](https://drive.google.com/uc?export=view&id=14BiPrb76ujujdP5mvqYlCJQYv70P7ulR)

Entonces, vamos a darle como input al modelo 3 valores secuenciales y esperar que la red genere una salida.

Se calcularán los valores de las unidades ocultas a los momentos de tiempo 1, 2 y 3. $h_0$ es inicializado con un vector de ceros. El output ($y_3$) es calculado utilizando $h_3$ y $w_y$. Veamos como funciona esto.

In [ ]:
# Creamos un array con 3 valores
x = np.array([1, 2, 3])
# Hacemos un reshape del input para obtener: sample_size x time_steps x features 
x_input = np.reshape(x,(1, 3, 1))
x_input

In [ ]:
y_pred_model = demo_model.predict(x_input)

m = 2
h0 = np.zeros(m)
h1 = np.dot(x[0], wx) + h0 + bh
h2 = np.dot(x[1], wx) + np.dot(h1,wh) + bh
h3 = np.dot(x[2], wx) + np.dot(h2,wh) + bh
o3 = np.dot(h3, wy) + by

print('h0 = ', h0)
print('h1 = ', h1)
print('h2 = ', h2)
print('h3 = ', h3)

print("Predicción de la red: ", y_pred_model)
print("Predicción de nuestro cálculo: ", o3)

# Ejemplo de Aplicación: Predicción del Precio de un Activo.

Veamos un caso de aplicación para tratar de predecir el precio de un activo. Para ello, seguiremos los siguientes pasos:

1. Lectura del Dataset
2. División de los datos en train y test
3. Preparación del input al formato requerido por Keras
4. Creación y entrenamiento de un modelo RNN
5. Obtención de las predicciones de train y test y los RMSE
6. Visualización de los resultados

## Pasos 1 y 2: Lectura y División de los Datos en Train y Test

La siguiente función, divide los datos en train y test, dada una serie de pandas con estructura temporal. Devuelve arrays de una dimensión para train y test.

Antes de utilizar esta función, escalaremos los datos entre 0 y 1, usando `MinMaxScaler` de Scikit-Learn.


In [ ]:
# Parameter split_percent defines the ratio of training examples
def get_train_test(dataframe, split_percent=0.8):
    data = np.array(dataframe.values.astype('float32'))
    n = len(data)
    # Point for splitting data into train and test
    split = int(n*split_percent)
    train_data = data[range(split)]
    test_data = data[split:]
    return train_data, test_data, data

Usando Yahoo Finance, podemos obtener el precio histórico de algún activo. En este caso, usaremos el precio de cierre de un año para Microsoft (ticker `MSFT`).

In [ ]:
msft = yf.Ticker("MSFT")

In [ ]:
msft_close = msft.history(period="1y")[['Close']]

In [ ]:
import plotly.express as px
px.line(data_frame=msft_close, x=msft_close.index, y='Close')

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(msft_close).flatten()

In [ ]:
# scaler.inverse_transform(msft_close)
msft_close['Close'] = data

In [ ]:
train_data, test_data, data = get_train_test(msft_close)

## Paso 3: Hacemos Reshape de la Data para Keras

En este paso, se preparan los datos para el entrenamiento del modelo en Keras. El *input array* debería tener la siguiente forma: `total_samples` x `time_steps` x `features`.

Hay muchas maneras de preparar datos de series temporales para el entrenamiento. En este caso, vamos a crear un input con *time steps* no superpuestos, tal como se puede ver en la siguiente imagen para 2 `time_steps`. Este parámetro indica la cantidad de valores $t$ a utilizar para predecir el valor $t+1$.

![img](https://drive.google.com/uc?export=view&id=14E-ny1p0lwvemV6WfSC1rYmIl4jZLXqw)

La función `get_XY()` toma un array unidimensional como input y lo 
convierte en los arrays de input X y target Y que necesitamos.
Probemos usando 10 `time_steps` (relacionado a la ciclicidad de los datos) y veamos qué ocurre.

In [ ]:
# Prepare the input X and target Y
def get_XY(dat, time_steps):
    # Indices of target array
    Y_ind = np.arange(time_steps, len(dat), time_steps)
    Y = dat[Y_ind]
    # Prepare X
    rows_x = len(Y)
    X = dat[range(time_steps*rows_x)]
    X = np.reshape(X, (rows_x, time_steps, 1))    
    return X, Y

In [ ]:
time_steps = 10
trainX, trainY = get_XY(train_data, time_steps)
testX, testY = get_XY(test_data, time_steps)

In [ ]:
trainX.shape, trainY.shape

## Paso 4: Crear un Modelo RNN y Entrenarlo

In [ ]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, 
                        activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
model = create_RNN(hidden_units=3, dense_units=1, input_shape=(time_steps,1), 
                   activation=['tanh', 'tanh'])
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)

## Paso 5: Calcular la Raíz del Error Cuadrático Medio (RMSE)

La función `print_error()` calcula la raíz el error cuadrático medio entre los valores reales y los predichos.


In [ ]:
def print_error(trainY, testY, train_predict, test_predict):    
    # Error of predictions
    train_rmse = math.sqrt(mean_squared_error(trainY, train_predict))
    test_rmse = math.sqrt(mean_squared_error(testY, test_predict))
    # Print RMSE
    print('Train RMSE: %.3f RMSE' % (train_rmse))
    print('Test RMSE: %.3f RMSE' % (test_rmse))    

# Obtener predicciones
train_predict = model.predict(trainX)
test_predict = model.predict(testX)
# Reescaladas
rescaled_train_predict = scaler.inverse_transform(model.predict(trainX))
rescaled_test_predict = scaler.inverse_transform(model.predict(testX))

rescaled_trainY = scaler.inverse_transform(trainY)
rescaled_testY = scaler.inverse_transform(testY)
# Mean square error
print('Con Scaling')
print_error(trainY, testY, train_predict, test_predict)
print('Sin Scaling')
print_error(rescaled_trainY, rescaled_testY, rescaled_train_predict, rescaled_test_predict)

## Paso 6: Visualizar los Resultados

La siguiente funcion grafica los valores reales de nuestra serie temporal y los predichos. La línea roja separa los datos de train y test.

In [ ]:
# Plot the result
def plot_result(trainY, testY, train_predict, test_predict):
    actual = np.append(trainY, testY)
    predictions = np.append(train_predict, test_predict)
    rows = len(actual)
    plt.figure(figsize=(15, 6), dpi=80)
    plt.plot(range(rows), actual)
    plt.plot(range(rows), predictions)
    plt.axvline(x=len(trainY), color='r')
    plt.legend(['Actual', 'Predictions'])
    plt.xlabel('Observation number after given time steps')
    plt.ylabel('Sunspots scaled')
    plt.title('Actual and Predicted Values. The Red Line Separates The Training And Test Examples')

In [ ]:
# plot_result(trainY, testY, train_predict, test_predict)
plot_result(rescaled_trainY, rescaled_testY, rescaled_train_predict, rescaled_test_predict)

# Otros Tipos más Sofisticados de RNN

Otro tipo de red recurrente, más sofisticado que la capa simple que vimos es la **LSTM** (**Long Short-Term Memory**), que viene a subsanar algunos inconvenientes de la red recurrente más simple (el problema de desvanecimiento del gradiente o *vanishing gradient* que afecta su performance).

A diferencia de las redes recurrentes tradicionales, una red neuronal *LSTM* está mejor adaptada para aprender de la experiencia a clasificar, procesar y predecir series temporales cuando hay retardos o *lags* de tiempos muy prolongados de tamaño desconocido entre eventos relevantes.

La habilidad de este algoritmo de tratar con retardos o *lags* muy prolongados tiene que ver con como hace la propagación hacia atrás del error (*Backpropagation*) de una forma constante.

Para más información, pueden consultar el siguiente material:

- https://www.altumintelligence.com/articles/a/Time-Series-Prediction-Using-LSTM-Deep-Neural-Networks
- https://machinelearningmastery.com/prepare-univariate-time-series-data-long-short-term-memory-networks/
- https://github.com/leriomaggio/deep-learning-keras-tensorflow/blob/master/7.%20Recurrent%20Neural%20Networks/7.1%20RNN%20and%20LSTM.ipynb
- https://towardsdatascience.com/the-most-intuitive-and-easiest-guide-for-recurrent-neural-network-873c29da73c7
- https://towardsdatascience.com/the-most-intuitive-and-easiest-guide-for-recurrent-neural-network-873c29da73c7